# Pinecone — Introduction (filled)

Denne notebooken var mest tomme code-celler. Jeg fylte den med en komplett “minimum demo”:

- opprette index
- koble til index
- upsert vectors
- query vectors

> Viktig: `dimension` må matche embedding-modellen du bruker (f.eks. 1536 for mange OpenAI-embeddings).


In [2]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os
from dotenv import load_dotenv, find_dotenv

In [3]:
load_dotenv(find_dotenv(), override = True)

True

In [4]:
pc = Pinecone(api_key = os.environ.get("PINECONE_API_KEY"), environment = os.environ.get("PINECONE_ENV"))

In [5]:
pc.list_indexes()

{'indexes': [{'dimension': 3,
              'host': 'my-index-bbthoyp.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'my-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [ ]:
# Create an index (serverless) if it doesn't exist

index_name = "course-index"
dimension = 1536  # must match your embedding model output size
metric = "cosine"

existing = [x["name"] for x in pc.list_indexes().get("indexes", [])]
if index_name not in existing:
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

pc.list_indexes()

In [ ]:
# Connect to the index
index = pc.Index(index_name)

# Basic health/stats
index.describe_index_stats()

In [ ]:
# Upsert a few dummy vectors (for testing)

vectors = [
    ("vec-1", [0.1] * dimension, {"source": "demo", "label": "a"}),
    ("vec-2", [0.2] * dimension, {"source": "demo", "label": "b"}),
]

index.upsert(vectors=vectors)
index.describe_index_stats()

In [ ]:
# Query the index (top_k nearest vectors)

query_vec = [0.15] * dimension
res = index.query(vector=query_vec, top_k=2, include_metadata=True)
res